## Preprocessing

In [1]:
# Import our dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME', as well as the 'STATUS' and 'SPECIAL_CONSIDERATIONS' columns.
application_df = application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'])
application_df.head(1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning.
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced.
# Use the variable name `application_types_to_replace`.
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 200].index.tolist()

# Replace in Dataframe.
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

# Check to make sure binning was successful.
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning.
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1.
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced.
# use the variable name `classifications_to_replace`.
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index.tolist()

# Replace in Dataframe.
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, 'Other')
    
# Check to make sure binning was successful.
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Create an empty list to hold our possible outliers.
outliers = []

# Calculate the IQR and quantitatively determine if there are any potential outliers in the 'ASK_AMT' column.
quartiles = application_df['ASK_AMT'].quantile([.25, .5, .75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq - lowerq 
lower_bound = lowerq - (1.5 * iqr)
upper_bound = upperq + (1.5 * iqr)

In [10]:
# Determine outliers using upper and lower bounds.
for i in application_df['ASK_AMT']:
    if (i < lower_bound) | (i > upper_bound):
        outliers.append(i)

# Print out all the possible outliers.
print(f'Count: {len(outliers)}, Potential outliers: {outliers}')

Count: 8206, Potential outliers: [108590, 142590, 31452, 7508025, 94389, 69656, 165593, 86380556, 376109, 318296, 44004, 323466, 709815, 1329410, 31613, 1549352, 577113, 44623, 64868, 131620, 306133, 66916, 236206, 393704, 9955466, 94869, 36607, 484855, 137272, 101980, 4410503, 15477, 24211, 638229, 20520, 34879, 205120, 12829, 452970, 72961, 12860, 696502, 259589, 49112, 86584, 4108874, 25175802, 132449, 73686, 723426, 521883, 855350, 370941, 1728949, 287332, 1790960, 23799, 201655, 27285052, 74643, 640921, 2705244, 2770474, 61686, 1064584, 25736, 17512616, 39064, 67654, 139694, 212362, 87981, 16027, 1066994, 107484, 20380, 293732, 2471994, 2793433, 324268, 343650, 497726, 28993, 118358, 2257305, 64723, 403069, 329188, 180240, 47901, 23322, 102050, 25846, 10935786, 1562336, 281725, 122937, 12753, 410463, 38286, 22527, 47129, 966422, 718228, 2461713, 876484, 14698, 110885, 64075, 78769, 2908673, 216643, 16221, 140950, 3676333, 30079, 98931, 12509, 20770, 118386, 120281, 71609, 155956, 

In [11]:
# Check length of original DataFrame before we drop columns with possible outliers.
len(application_df)

34299

In [12]:
# Identify outlier amounts with a 'DROP' label and then drop these rows from the DataFrame.
for amt in outliers:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt, 'DROP')

application_df = application_df.drop(application_df[application_df['ASK_AMT'] == 'DROP'].index)

In [13]:
# Check to see that 8206 rows were dropped.
len(application_df)

26093

In [14]:
# Convert 'ASK_AMT' column back to integer type.
application_df['ASK_AMT'] = application_df['ASK_AMT'].astype(int)

In [15]:
# Check data types.
application_df.dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [16]:
# Create a list of column names from application_df that hold categorical data.
cat_data = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()

# Convert categorical data to numeric with `pd.get_dummies`.
dummies = pd.get_dummies(application_df[cat_data])
dummies.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [17]:
# Merge the dummies data with the original application_df.
application_df = application_df.merge(dummies, left_index=True, right_index=True)

# Drop the old categorical columns that are not obsolete because of the dummy data.
application_df = application_df.drop(columns=cat_data)

In [18]:
# Check columns.
application_df.columns

Index(['ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       

In [19]:
# Split our preprocessed data into our features and target arrays.
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [20]:
# Create a StandardScaler instances.
scaler = StandardScaler()

# Fit the StandardScaler.
X_scaler = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer.
nn.add(tf.keras.layers.Dense(units=16, activation='relu', input_dim=40))

# Second hidden layer.
nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer.
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model.
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                656       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 945
Trainable params: 945
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model.
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# Train the model.
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
612/612 [==============================] - 2s 2ms/step - loss: 0.5765 - accuracy: 0.7236
Epoch 2/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5405 - accuracy: 0.7438
Epoch 3/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5346 - accuracy: 0.7464
Epoch 4/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5312 - accuracy: 0.7479
Epoch 5/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5298 - accuracy: 0.7491
Epoch 6/50
612/612 [==============================] - 2s 3ms/step - loss: 0.5284 - accuracy: 0.7501
Epoch 7/50
612/612 [==============================] - 2s 3ms/step - loss: 0.5274 - accuracy: 0.7498
Epoch 8/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5263 - accuracy: 0.7523
Epoch 9/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5261 - accuracy: 0.7506
Epoch 10/50
612/612 [==============================] - 1s 2ms/step - loss: 0.5252 - accuracy: 0.7519

In [24]:
# Evaluate the model using the test data.
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

204/204 - 0s - loss: 0.5416 - accuracy: 0.7394 - 418ms/epoch - 2ms/step
Loss: 0.5416107773780823, Accuracy: 0.7394236922264099


In [25]:
# Export our model to HDF5 file.
nn.save('AlphabetSoupCharity_Optimization_4.h5')